In [ ]:
from inference import get_model
#from google.colab import userdata

ROBOFLOW_API_KEY = "ITfUpuY5QO9WTBpcEXTh"  # Replace with your actual Roboflow API key

PLAYER_DETECTION_MODEL_ID = "football-players-detection-3zvbc/12"
PLAYER_DETECTION_MODEL = get_model(model_id=PLAYER_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

In [ ]:
SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/videos/121364_0.mp4"


In [ ]:
import supervision as sv
from TeamClassifier import TeamClassifier
import torch
import cv2
import numpy as np
from tqdm import tqdm 

In [ ]:



DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
STRIDE = 30 

BALL_ID = 0
PLAYER_ID = 2

crops = extract_crops(SOURCE_VIDEO_PATH)

team_classifier = TeamClassifier(device=DEVICE)
team_classifier.fit(crops)

ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)

tracker = sv.ByteTrack()
tracker.reset()

In [ ]:

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
detections = sv.Detections.from_inference(result)

ball_detections = detections[detections.class_id == BALL_ID]
ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

all_detections = detections[detections.class_id != BALL_ID]
all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)
all_detections = tracker.update_with_detections(detections=all_detections)

players_detections = all_detections[all_detections.class_id == PLAYER_ID]
players_crops = [sv.crop_image(frame,xyxy) for xyxy in players_detections.xyxy]
players_detections.class_id = team_classifier.predict(players_crops)

labels = [
    f"#{tracker_id}"
    for tracker_id
    in players_detections.tracker_id
]

annotated_frame = frame.copy()
annotated_frame = ellipse_annotator.annotate( scene=annotated_frame, detections = players_detections)
annotated_frame = label_annotator.annotate(scene=annotated_frame,detections = players_detections, labels=labels)
annotated_frame = triangle_annotator.annotate( scene=annotated_frame, detections = ball_detections)

sv.plot_image(annotated_frame)

In [ ]:
def extract_crops(source_video_path : str):
        frame_generator = sv.get_video_frames_generator(source_video_path, stride=STRIDE)##Getting the frames with a stride of 30

        crops = []

        for frame in tqdm(frame_generator , desc="Extracting crops"):
            result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.5)[0]
            detections = sv.Detections.from_inference(result)
            detections = detections.with_nms(threshold=0.5 , class_agnostic=True)
            detections = detections[detections.class_id == PLAYER_ID]

            crops += [
                sv.crop_image(frame,xyxy)
                for xyxy
                in detections.xyxy
            ]
        return crops

In [ ]:
from collections import defaultdict
from TeamClassifier import TeamClassifier
import torch

def track_ball_possession(source_video_path: str):
    frame_generator = sv.get_video_frames_generator(source_video_path, stride=STRIDE)
    
    possession_tracker = defaultdict(int)  # {team_id: frame_count}
    last_team = None  # Keep track of which team had possession in the last frame
    consecutive_frames = 0  # Track how long the same team has possession

    for i, frame in enumerate(tqdm(frame_generator, desc="Tracking ball possession")):
        if i >= 3:  # Process only the first 3 frames
            break

        result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.5)[0]
        detections = sv.Detections.from_inference(result)
        
        # Separate ball and players detections
        ball_detections = detections[detections.class_id == BALL_ID]
        players_detections = detections[detections.class_id == PLAYER_ID]

        if len(ball_detections) == 0 or len(players_detections) == 0:
            continue  # Skip frame if no ball or players detected

        ball_center = get_center(ball_detections.xyxy[0])  # Assuming one ball

        crops = extract_crops(SOURCE_VIDEO_PATH)
        team_classifier = TeamClassifier(device=DEVICE)
        team_classifier.fit(crops)
        
        # Assign team to players
        players_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
        players_detections.class_id = team_classifier.predict(players_crops)  # Assign team ID

        # Find closest player to ball
        closest_player, closest_distance = None, float("inf")
        for player, xyxy in zip(players_detections.class_id, players_detections.xyxy):
            player_center = get_center(xyxy)
            distance = get_distance(ball_center, player_center)
            if distance < closest_distance:
                closest_distance = distance
                closest_player = player  # This will be the team ID

        if closest_player is not None:
            if closest_player == last_team:
                consecutive_frames += 1  # Increment possession time
            else:
                if last_team is not None:
                    possession_tracker[last_team] += consecutive_frames
                last_team = closest_player
                consecutive_frames = 1  # Reset count for new team

    # Store the last team's possession count
    if last_team is not None:
        possession_tracker[last_team] += consecutive_frames

    return possession_tracker  # Dictionary of {team_id: possession_frames}


In [ ]:
import numpy as np


def get_center(xyxy):
    """Calculate the center (x, y) of a bounding box."""
    x_min, y_min, x_max, y_max = xyxy
    return ((x_min + x_max) / 2, (y_min + y_max) / 2)


def get_distance(p1, p2):
    """Calculate the Euclidean distance between two points."""
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)


In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
STRIDE = 30
BALL_ID = 0
PLAYER_ID = 2
possession = track_ball_possession(SOURCE_VIDEO_PATH)

for team_id, frames in possession.items():
    print(f"Team {team_id} had possession for {frames} frames.")
